In [24]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
%matplotlib inline


In [25]:
#Scraping a list on Goodreads

In [26]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [27]:
my_url = 'https://www.goodreads.com/list/show/127078.2019_Books_by_Authors_of_Color'
list_df = pd.DataFrame

def get_list(my_df, my_url, page=1):
    
    
    
    #check for list page number:
    if page <= 0:
        print('Imput Error')
        return
    if page > 1:
        url = my_url + '?page=' + str(page)
    else:
        url = my_url
    
    #open the page
    html = urlopen(url)
    soup = BeautifulSoup(html, 'lxml')
    another_list=[]
    author_list=[]
    
    # get goodreads author links:
    results = soup.find_all('a', {'class': ['authorName']})
    for result in results:
        another_list.append(str(result))
    for link in another_list:
        author_url = link
        regex = r'/\d.*?\"'
        authors = re.findall(regex,link)
        for author in authors:
            author = author[1:-1]
            author = author.replace("_"," ")
            author = author.split('.')
            author.append(author_url)
            author_list.append(author)
            
    # prepare the df to be returned
    headers = ['ID', 'Name', 'Goodreads_Page']
    if my_df.empty:
        df = pd.DataFrame(author_list, columns = headers)
    else:
        af = pd.DataFrame(author_list, columns = headers)
        frames = [my_df, af] 
        df = pd.DataFrame(pd.concat(frames))
    # get next page, if any:
    next_page = None
    next_page = soup.find_all("a", {'class': ['next_page']}, href=lambda href: href and "show" in href)
    # save file, or go recursive.
    if next_page == []:
        regex = r'\d.*'
        filename = (re.findall(regex,my_url))
        df.to_csv(str('goodreads/'+filename[0]+'.csv'), index = False)
        print('Saved to disk as ', ('goodreads/'+str(filename[0]+'.csv')))
        
    else:
        get_list(df, my_url, page+1)

In [31]:
 #get_list(list_df, my_url)

In [36]:
# start by performing a search on 'listopia', then insert here the results page url:'https://www.goodreads.com'
base_url = 'https://www.goodreads.com'
search_url = 'https://www.goodreads.com/search?utf8=%E2%9C%93&search_type=lists&q=poc'

def scrap_list(search_url, passed_list=[]):
    list_url = search_url
    
    #open the page and get some jars ready for your jam
    html = urlopen(list_url)
    list_soup = BeautifulSoup(html, 'lxml')
    first_list=[]
    bunch_of_lists=[]
    
    # look for each item in page (it's a list of lists)
    my_lists = list_soup.find_all('a', {'class': ['listTitle']})
    
    for a_list in my_lists:
        first_list.append(str(a_list))
  
    for a_link in first_list:
        regex = r"href=\".+?\">"
        linko = re.findall(regex,a_link)
        linko = linko[0][6:-2]
        linko = base_url+linko
        bunch_of_lists.append(linko)
        
    # now that we went through the page, 
    # let's look for other pages.
   
    passed_list.extend(bunch_of_lists)
    
    # get next page, if any:
    next_page = None
    next_page = list_soup.find('a', {'class': ['next_page']})
    
    # store list if done. else, go recursive.
    if next_page != None:
        texta = str(next_page)
        regexa = r"href=\".*?\">"
        linka = re.findall(regexa,texta)
        linka = linka[0]
        linka = linka[6:-13]
        linka = base_url+linka
        scrap_list(linka, passed_list)
    else:
        my_sr = pd.Series(passed_list)
        my_sr.to_csv('goodreads/mysearch.csv')
        for lista in passed_list:
            list_df = pd.DataFrame()
            get_list(list_df, lista)
            print('Progress: ', i, ' out of ', len(passed_list), ' lists saved.')
        print('Done.')
        return

In [35]:
scrap_list(search_url)

Saved to disk as  goodreads/11395.Non_Caucasian_Protagonists_in_Science_Fiction_Fantasy_Horror_and_Paranormal_Romance.csv
Done.
Saved to disk as  goodreads/73078.Diversity_in_Fantasy_and_Science_Fiction.csv
Done.
Saved to disk as  goodreads/27758.Anticipated_Literary_Reads_for_Readers_of_Color_2013.csv
Done.
Saved to disk as  goodreads/107430.Diversity_Bingo_2017.csv
Done.
Saved to disk as  goodreads/88553.2017_YA_MG_Books_With_POC_Leads.csv
Done.
Saved to disk as  goodreads/12716.Speculative_Fiction_by_Authors_of_Color.csv
Done.
Saved to disk as  goodreads/127078.2019_Books_by_Authors_of_Color.csv
Done.
Saved to disk as  goodreads/96059._1000BlackGirlBooks_Suggestions_for_Marley_Dias_s_Amazing_Book_Drive.csv
Done.
Saved to disk as  goodreads/3446.African_American_Books_for_Teens.csv
Done.
Saved to disk as  goodreads/99680.2018_YA_MG_Books_With_POC_Leads_.csv
Done.
Saved to disk as  goodreads/119033.Anticipated_Literary_Reads_For_Readers_of_Color_2018.csv
Done.
Saved to disk as  goodre